hist, new, total, new/hist で特徴量作成

https://www.kaggle.com/code/raddar/target-true-meaning-revealed

2**target

amountを調整

np.round(df['purchase_amount'] / 0.00150265118 + 497.06)

In [1]:
import warnings
import time
import sys
import datetime
import pickle
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)

%matplotlib inline

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
def preprocessing(df):
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    for col in ['category_3']:
        df[col] = df[col].map({'A':1, 'B':2,'C':3})
    return df

def read_data(input_file):
    df = pd.read_csv(input_file)
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['elapsed_time'] = (pd.Timestamp('2018-02-01') - df['first_active_month']).dt.days
    return df

In [4]:
# データの読み込み
path_new = '../../../../data/raw/new_merchant_transactions.csv'
path_hist = '../../../../data/raw/historical_transactions.csv'
path_train = '../../../../data/raw/train.csv'
path_test = '../../../../data/raw/test.csv'

new = pd.read_csv(path_new, parse_dates=['purchase_date'])
hist = pd.read_csv(path_hist ,parse_dates=['purchase_date'])

new = preprocessing(new)
hist = preprocessing(hist)

new = reduce_mem_usage(new)
hist = reduce_mem_usage(hist)

train = pd.read_csv(path_train, parse_dates=['first_active_month'])
test = pd.read_csv(path_test, parse_dates=['first_active_month'])

target = train['target']
#del train['target']

Mem. usage decreased to 76.76 Mb (63.4% reduction)
Mem. usage decreased to 1193.84 Mb (61.6% reduction)


In [21]:
train = pd.read_csv(path_train, parse_dates=['first_active_month'])
test = pd.read_csv(path_test, parse_dates=['first_active_month'])

## train, test 処理

In [22]:
#train['target'] = 2**train['target']

In [5]:
test_null_id = test[test.first_active_month.isna()].card_id.iloc[-1]
null_month = hist[hist.card_id == test_null_id].purchase_date.dt.strftime("%Y-%m-01").min()
test.fillna({'first_active_month':null_month},inplace=True)

In [6]:
for df in [test, train]:
    # https://www.kaggle.com/code/mfjwr1/simple-lightgbm-without-blending
    df['elapsed_time'] = (pd.Timestamp('2018-02-01') - df['first_active_month']).dt.days
    df['quarter'] = df['first_active_month'].dt.quarter
    df['days_feature1'] = df['elapsed_time'] * df['feature_1']
    df['days_feature2'] = df['elapsed_time'] * df['feature_2']
    df['days_feature3'] = df['elapsed_time'] * df['feature_3']
    df['days_feature1_ratio'] = df['feature_1'] / df['elapsed_time']
    df['days_feature2_ratio'] = df['feature_2'] / df['elapsed_time']
    df['days_feature3_ratio'] = df['feature_3'] / df['elapsed_time']
    df['feature_sum'] = df['feature_1'] + df['feature_2'] + df['feature_3']
    df['feature_mean'] = df['feature_sum']/3
    df['feature_max'] = df[['feature_1', 'feature_2', 'feature_3']].max(axis=1)
    df['feature_min'] = df[['feature_1', 'feature_2', 'feature_3']].min(axis=1)
    df['feature_var'] = df[['feature_1', 'feature_2', 'feature_3']].std(axis=1)

In [7]:
for df in [test,train]:
    df['first_active_month']  = (df['first_active_month'] - df['first_active_month'].min()).dt.days

## 特徴量作成

In [8]:
for df in [hist, new]:
    df['purchase_amount'] = np.round(df['purchase_amount'] / 0.00150265118 + 497.06)

In [11]:
for df in [hist, new]:
    df['category_2'].fillna(1,inplace=True) # 1が最頻値
    df['category_3'].fillna(1,inplace=True) # 1が最頻値
    df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True) # 最頻値

    # https://www.kaggle.com/code/mfjwr1/simple-lightgbm-without-blending
    df['installments'].replace(-1, np.nan,inplace=True)
    df['installments'].replace(999, np.nan,inplace=True)
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['month'] = df['purchase_date'].dt.month
    df['day'] = df['purchase_date'].dt.day
    df['hour'] = df['purchase_date'].dt.hour
    #df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['weekday'] = df['purchase_date'].dt.weekday
    df['weekend'] = (df['purchase_date'].dt.weekday >=5).astype(int)
    # additional features
    df['price'] = df['purchase_amount'] / df['installments']
    # additional features
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    df['duration'] = df['purchase_amount']*df['month_diff']
    df['amount_month_ratio'] = df['purchase_amount']/df['month_diff']

In [23]:
hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29112361 entries, 0 to 29112360
Data columns (total 23 columns):
 #   Column                Dtype         
---  ------                -----         
 0   authorized_flag       int8          
 1   card_id               object        
 2   city_id               int16         
 3   category_1            int8          
 4   installments          float64       
 5   category_3            float16       
 6   merchant_category_id  int16         
 7   merchant_id           object        
 8   month_lag             int8          
 9   purchase_amount       float32       
 10  purchase_date         datetime64[ns]
 11  category_2            float16       
 12  state_id              int8          
 13  subsector_id          int8          
 14  month                 int32         
 15  day                   int32         
 16  hour                  int32         
 17  weekday               int32         
 18  weekend               int32         
 19

In [12]:
# 承認フラグ　Yes,Noの数と比
# histのみ（newはYesのみのため）
authorized_flag = hist[['card_id','authorized_flag']].groupby(['card_id','authorized_flag']).agg(count=('card_id','count')).reset_index()

# カードIDごとの合計を計算
total_counts = authorized_flag.groupby('card_id')['count'].sum().reset_index()
total_counts = total_counts.rename(columns={'count': 'total_count'})

# 元のデータと合計を結合
authorized_flag = authorized_flag.merge(total_counts, on='card_id')

# 割合を計算
authorized_flag['ratio'] = authorized_flag['count'] / authorized_flag['total_count']
#hist[['card_id','authorized_flag']].groupby(['card_id']).agg(count=('authorized_flag','mean')).reset_index()

# 結果を整形
authorized_ratio = authorized_flag.pivot(index='card_id', columns='authorized_flag', values=['ratio','count']).reset_index().rename(columns={0: 'No', 1: 'Yes'})
authorized_ratio.columns = ['_'.join(col).strip() for col in authorized_ratio.columns.values]
authorized_ratio.rename(columns={'card_id_': 'card_id'},inplace=True)

In [13]:
total = pd.concat([hist,new],axis=0).reset_index(drop=True)

In [25]:
for df in [hist, new, total]:
    df_tmp = df.groupby('card_id',as_index=False)\
        .agg(
            frequency = ('card_id','count'),
            amount_total = ('purchase_amount','sum'),
            amount_mean = ('purchase_amount','mean'),
            amount_max = ('purchase_amount','max'),
            amount_min = ('purchase_amount','min'),
            amount_median = ('purchase_amount','median'),
            merchant_category_nu = ('merchant_category_id','nunique'),
            merchant_category_mode = ('merchant_category_id',lambda x: x.mode()[0]),
            city_nu = ('city_id','nunique'),
            city_mode = ('city_id',lambda x: x.mode()[0]),
            state_nu = ('state_id','nunique'),
            state_mode = ('state_id',lambda x: x.mode()[0]),
            date_min  = ('purchase_date','min'),
            date_max  = ('purchase_date','max'),
            installments_mean = ('installments','mean'),
            installments_max = ('installments','max'),
            installments_min = ('installments','min'),
            installments_median = ('installments','median'),
            installments_sum = ('installments','sum'),
            month_lag_mean = ('month_lag','mean'),
            month_lag_max = ('month_lag','max'),
            month_lag_min = ('month_lag','min'),
            month_lag_median = ('month_lag','median'),
            category_1_mod = ('category_1',lambda x: x.mode()[0]),
            category_2_mod = ('category_2',lambda x: x.mode()[0]),
            category_3_mod = ('category_3',lambda x: x.mode()[0]),
            # additional
            weekend_mean = ('weekend', 'mean'),
            weekday_mean = ('weekday', 'mean'),
            day_nu = ('day', 'nunique'),
            day_min = ('day', 'min'),
            day_max = ('day', 'max'),
            price_sum = ('price', 'sum'),
            price_min = ('price', 'min'),
            price_max = ('price', 'max'),
            price_mean = ('price', 'mean'),
            duration_min = ('duration', 'min'),
            duration_max = ('duration', 'max'),
            duration_mean = ('duration', 'mean'),
            amount_month_ratio_min = ('amount_month_ratio', 'min'),
            amount_month_ratio_max = ('amount_month_ratio', 'max'),
            amount_month_ratio_mean = ('amount_month_ratio', 'mean')
        )

    df_tmp['duration'] = (df_tmp['date_max'] - df_tmp['date_min']).dt.days
    df_tmp.date_max = (df_tmp.date_max - df_tmp.date_min.min()).dt.days
    df_tmp.date_min = (df_tmp.date_min - df_tmp.date_min.min()).dt.days

    df_tmp.merchant_category_mode = pd.to_numeric(df_tmp.merchant_category_mode, errors='coerce')
    if df == hist:
        df_tmp = df_tmp.merge(authorized_ratio,on='card_id')
        df_tmp.columns = ['hist_' + col if col != 'card_id' else col for col in df_tmp.columns]

SyntaxError: keyword argument repeated: duration_min (1594081718.py, line 40)

In [26]:
hist_id_u = hist.groupby('card_id',as_index=False)\
    .agg(
        frequency = ('card_id','count'),
        amount_total = ('purchase_amount','sum'),
        amount_mean = ('purchase_amount','mean'),
        amount_max = ('purchase_amount','max'),
        amount_min = ('purchase_amount','min'),
        amount_median = ('purchase_amount','median'),
        merchant_category_nu = ('merchant_category_id','nunique'),
        merchant_category_mode = ('merchant_category_id',lambda x: x.mode()[0]),
        city_nu = ('city_id','nunique'),
        city_mode = ('city_id',lambda x: x.mode()[0]),
        state_nu = ('state_id','nunique'),
        state_mode = ('state_id',lambda x: x.mode()[0]),
        date_min  = ('purchase_date','min'),
        date_max  = ('purchase_date','max'),
        installments_mean = ('installments','mean'),
        installments_max = ('installments','max'),
        installments_min = ('installments','min'),
        installments_median = ('installments','median'),
        installments_sum = ('installments','sum'),
        month_lag_mean = ('month_lag','mean'),
        month_lag_max = ('month_lag','max'),
        month_lag_min = ('month_lag','min'),
        month_lag_median = ('month_lag','median'),
        category_1_mod = ('category_1',lambda x: x.mode()[0]),
        category_2_mod = ('category_2',lambda x: x.mode()[0]),
        category_3_mod = ('category_3',lambda x: x.mode()[0]),
        # additional
        weekend_mean = ('weekend', 'mean'),
        weekday_mean = ('weekday', 'mean'),
        day_nu = ('day', 'nunique'),
        day_min = ('day', 'min'),
        day_max = ('day', 'max'),
        price_sum = ('price', 'sum'),
        price_min = ('price', 'min'),
        price_max = ('price', 'max'),
        price_mean = ('price', 'mean'),
        duration_min = ('duration', 'min'),
        duration_max = ('duration', 'max'),
        duration_mean = ('duration', 'mean'),
        amount_month_ratio_min = ('amount_month_ratio', 'min'),
        amount_month_ratio_max = ('amount_month_ratio', 'max'),
        amount_month_ratio_mean = ('amount_month_ratio', 'mean')
    )

In [27]:
hist_id_u = hist_id_u.merge(authorized_ratio,on='card_id')

In [28]:
hist_id_u['duration'] = (hist_id_u['date_max'] - hist_id_u['date_min']).dt.days
hist_id_u.date_max = (hist_id_u.date_max - hist_id_u.date_min.min()).dt.days
hist_id_u.date_min = (hist_id_u.date_min - hist_id_u.date_min.min()).dt.days

hist_id_u.merchant_category_mode = pd.to_numeric(hist_id_u.merchant_category_mode, errors='coerce')

hist_id_u.columns = ['hist_' + col if col != 'card_id' else col for col in hist_id_u.columns]

In [29]:
new_id_u = new.groupby('card_id',as_index=False)\
    .agg(
        frequency = ('card_id','count'),
        amount_total = ('purchase_amount','sum'),
        amount_mean = ('purchase_amount','mean'),
        amount_max = ('purchase_amount','max'),
        amount_min = ('purchase_amount','min'),
        amount_median = ('purchase_amount','median'),
        merchant_category_nu = ('merchant_category_id','nunique'),
        merchant_category_mode = ('merchant_category_id',lambda x: x.mode()[0]),
        city_nu = ('city_id','nunique'),
        city_mode = ('city_id',lambda x: x.mode()[0]),
        state_nu = ('state_id','nunique'),
        state_mode = ('state_id',lambda x: x.mode()[0]),
        date_min  = ('purchase_date','min'),
        date_max  = ('purchase_date','max'),
        installments_mean = ('installments','mean'),
        installments_max = ('installments','max'),
        installments_min = ('installments','min'),
        installments_median = ('installments','median'),
        installments_sum = ('installments','sum'),
        month_lag_mean = ('month_lag','mean'),
        month_lag_max = ('month_lag','max'),
        month_lag_min = ('month_lag','min'),
        month_lag_median = ('month_lag','median'),
        category_1_mod = ('category_1',lambda x: x.mode()[0]),
        category_2_mod = ('category_2',lambda x: x.mode()[0]),
        category_3_mod = ('category_3',lambda x: x.mode()[0]),
        # additional
        weekend_mean = ('weekend', 'mean'),
        weekday_mean = ('weekday', 'mean'),
        day_nu = ('day', 'nunique'),
        day_min = ('day', 'min'),
        day_max = ('day', 'max'),
        price_sum = ('price', 'sum'),
        price_min = ('price', 'min'),
        price_max = ('price', 'max'),
        price_mean = ('price', 'mean'),
        duration_min = ('duration', 'min'),
        duration_max = ('duration', 'max'),
        duration_mean = ('duration', 'mean'),
        amount_month_ratio_min = ('amount_month_ratio', 'min'),
        amount_month_ratio_max = ('amount_month_ratio', 'max'),
        amount_month_ratio_mean = ('amount_month_ratio', 'mean')
    )

In [30]:
new_id_u['duration'] = (new_id_u['date_max'] - new_id_u['date_min']).dt.days
new_id_u.date_max = (new_id_u.date_max - new_id_u.date_min.min()).dt.days
new_id_u.date_min = (new_id_u.date_min - new_id_u.date_min.min()).dt.days

new_id_u.merchant_category_mode = pd.to_numeric(new_id_u.merchant_category_mode, errors='coerce')

new_id_u.columns = ['new_' + col if col != 'card_id' else col for col in new_id_u.columns]

In [31]:
total_id_u = total.groupby('card_id',as_index=False)\
    .agg(
        frequency = ('card_id','count'),
        amount_total = ('purchase_amount','sum'),
        amount_mean = ('purchase_amount','mean'),
        amount_max = ('purchase_amount','max'),
        amount_min = ('purchase_amount','min'),
        amount_median = ('purchase_amount','median'),
        merchant_category_nu = ('merchant_category_id','nunique'),
        merchant_category_mode = ('merchant_category_id',lambda x: x.mode()[0]),
        city_nu = ('city_id','nunique'),
        city_mode = ('city_id',lambda x: x.mode()[0]),
        state_nu = ('state_id','nunique'),
        state_mode = ('state_id',lambda x: x.mode()[0]),
        date_min  = ('purchase_date','min'),
        date_max  = ('purchase_date','max'),
        installments_mean = ('installments','mean'),
        installments_max = ('installments','max'),
        installments_min = ('installments','min'),
        installments_median = ('installments','median'),
        installments_sum = ('installments','sum'),
        month_lag_mean = ('month_lag','mean'),
        month_lag_max = ('month_lag','max'),
        month_lag_min = ('month_lag','min'),
        month_lag_median = ('month_lag','median'),
        category_1_mod = ('category_1',lambda x: x.mode()[0]),
        category_2_mod = ('category_2',lambda x: x.mode()[0]),
        category_3_mod = ('category_3',lambda x: x.mode()[0]),
        # additional
        weekend_mean = ('weekend', 'mean'),
        weekday_mean = ('weekday', 'mean'),
        day_nu = ('day', 'nunique'),
        day_min = ('day', 'min'),
        day_max = ('day', 'max'),
        price_sum = ('price', 'sum'),
        price_min = ('price', 'min'),
        price_max = ('price', 'max'),
        price_mean = ('price', 'mean'),
        duration_min = ('duration', 'min'),
        duration_max = ('duration', 'max'),
        duration_mean = ('duration', 'mean'),
        amount_month_ratio_min = ('amount_month_ratio', 'min'),
        amount_month_ratio_max = ('amount_month_ratio', 'max'),
        amount_month_ratio_mean = ('amount_month_ratio', 'mean')
    )

In [32]:
total_id_u['duration'] = (total_id_u['date_max'] - total_id_u['date_min']).dt.days
total_id_u.date_max = (total_id_u.date_max - total_id_u.date_min.min()).dt.days
total_id_u.date_min = (total_id_u.date_min - total_id_u.date_min.min()).dt.days

total_id_u.merchant_category_mode = pd.to_numeric(total_id_u.merchant_category_mode, errors='coerce')

total_id_u.columns = ['total_' + col if col != 'card_id' else col for col in total_id_u.columns]

In [33]:
train = train.merge(hist_id_u, on='card_id',how='left')
test = test.merge(hist_id_u, on='card_id',how='left')

train = train.merge(new_id_u, on='card_id',how='left')
test = test.merge(new_id_u, on='card_id',how='left')

train = train.merge(total_id_u, on='card_id',how='left')
test = test.merge(total_id_u, on='card_id',how='left')

In [36]:
for df in [train, test]:
    df['hist_count_No'].fillna(0,inplace=True)
    df['hist_ratio_No'].fillna(0,inplace=True)
    df['new_amount_total'].fillna(0,inplace=True)
    df['new_amount_min'].fillna(0,inplace=True)
    df['new_amount_max'].fillna(0,inplace=True)
    df['new_amount_mean'].fillna(0,inplace=True)
    df['new_amount_median'].fillna(0,inplace=True)
    df['new_merchant_category_nu'].fillna(0,inplace=True)
    df['new_city_nu'].fillna(0,inplace=True)
    df['new_state_nu'].fillna(0,inplace=True)
    df['new_installments_sum'].fillna(0,inplace=True)
    df['new_installments_min'].fillna(0,inplace=True)
    df['new_installments_max'].fillna(0,inplace=True)
    df['new_installments_mean'].fillna(0,inplace=True)
    df['new_installments_median'].fillna(0,inplace=True)
    df['new_price_sum'].fillna(0,inplace=True)
    df['new_price_min'].fillna(0,inplace=True)
    df['new_price_max'].fillna(0,inplace=True)
    df['new_price_mean'].fillna(0,inplace=True)
    #df.fillna(0, inplace=True)

In [37]:
for df in [train, test]:
    df['ratio_amount_total'] = df['new_amount_total'] / df['hist_amount_total']
    df['ratio_amount_mean'] = df['new_amount_mean'] / df['hist_amount_mean']
    df['ratio_merchant_category_nu'] = df['new_merchant_category_nu'] / df['hist_merchant_category_nu']
    #df['ratio_installments_mean'] = df['new_installments_mean'] / df['hist_installments_mean']
    #df['ratio_installments_sum'] = df['new_installments_sum'] / df['hist_installments_sum']

In [38]:
path_train_prepro = '../../../../data/processed/train_processed_30.csv'
path_test_preprp = '../../../../data/processed/test_processed_30.csv'

train.to_csv(path_train_prepro,index=False)
test.to_csv(path_test_preprp,index=False)